<a href="https://colab.research.google.com/github/Ananya-Joshi/AutoPrompt/blob/main/AutoPrompt_%5BPublic_Facing%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers
!pip install --upgrade openai langchain

In [ ]:
content_variability_max = 30
max_char = 500
n = 100
resp_list = []

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from collections.abc import Iterator
from openai import OpenAI
import random
import pandas as pd
import json
import numpy as np
import re
from numpy.random import choice
#Demographics Pulled from US Census Data
client = OpenAI(api_key="YOUR API KEY")
gender_dist = {"man":0.5, "woman":0.5}
age_dist = {'child':0.05, 'teen':0.16, 'adult':0.62, 'elderly':0.17}
race_dist = {"black": 0.12, "white":0.63, "asian":0.04, "hispanic":0.16, "native":0.007, "":0.043}
geospatial_dist = {"southern U.S":0.25, "northern U.S":0.25, "eastern U.S":0.25, "western U.S":0.25}

#Generate a set of personas based on census data
personas = []
for i in range(100):
  personas.append([choice(list(age_dist.keys()), p=list(age_dist.values())),
                      choice(list(race_dist.keys()), p=list(race_dist.values())),
                      choice(list(gender_dist.keys()), p=list(gender_dist.values())),
                      choice(list(geospatial_dist.keys()), p=list(geospatial_dist.values()))
                      ])
ref = pd.DataFrame(personas).replace(age_dist).replace(gender_dist).replace(geospatial_dist).replace(race_dist)

In [ ]:
#Generate a set of 10 prompts from ChatGPT
ten_prompts = f"Think of 10 subjective cultural questions people ask ChatGPT. Output a prompt to ChatGPT per question with no commas that is less than 5 words and output as a .csv"
response1 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": f"{ten_prompts}"},
      ], seed=2
  )
questions = pd.DataFrame(response1.choices[0].message.content.split('\n'))
questions_list = list(map(lambda x: re.sub(r'[0-9].', ' ', x), questions[0]))
# questions_list =list(map(lambda x: x.replace('.,', '?,').split('?,')[1], questions_list[1:]))
# questions_list = questions_list[::3][1:]

In [ ]:
#Generate typical responses
responses = []
max_char = 200
for question in questions_list:
  response_quest = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": f"Let's play a game. You are role-playing a typical American. Output only the response to the following question for youreslf in less than {max_char} characters without referencing yourself at all. Question: {question}  "},
      ], seed=0
  )
  responses.append(response_quest.choices[0].message.content)

In [ ]:
#Personas rate typical responses
dict_list = []

for choices_list in personas[:35]:
        seed = int(random.random()*(25000))

        context_simulation = f"""Let's play a game. You are a {choices_list[0]} \
                          {choices_list[1]} {choices_list[2]} American. \
                          Complete the following 2 tasks,output as .json:\
                          Task #1: Create a detailed backstory about your background and print it. The json key for task #1 should be 'backstory'. \
                          Task #2: Think of your response to the following 10 questions with no commas in less than {max_char} character and output them as a dictionary. The json key for task #1 should be 'responses'. \
                          Q1: Recommend {questions_list[0]} \
                          Q2: Recommend {questions_list[1]} \
                          Q3: Recommend  {questions_list[2]} \
                          Q4: Recommend  {questions_list[3]} \
                          Q5: Recommend  {questions_list[4]} \
                          Q6: Recommend  {questions_list[5]} \
                          Q7: Recommend  {questions_list[6]} \
                          Q8: Recommend  {questions_list[7]} \
                          Q9: Recommend  {questions_list[8]} \
                          Q10: Recommend  {questions_list[9]} \
                           """

        response1 = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
              {"role": "user", "content": f"{context_simulation}"},
                ], seed=seed
            )
        try:
          with open(f"""e2_persona_{'_'.join(choices_list)}_seed_{seed}.json""", 'w') as f:
                json.dump(response1.choices[0].message.content, f)
          orig_dict = json.loads(response1.choices[0].message.content)['responses']

          context2 = f"""For the next 10 questions, rank how similar the 2 responses are from 0 (not similar) to 10 (same response). \
           Output as list (e.g. [1,0,2,0,3,4,5,6,7,2]): \
           #1: Response 1: {orig_dict['Q1']} and Response 2: {responses[0]} \
           #2: Response 1: {orig_dict['Q2']} and Response 2: {responses[1]} \
           #3: Response 1: {orig_dict['Q3']} and Response 2: {responses[2]} \
           #4: Response 1: {orig_dict['Q4']} and Response 2: {responses[3]} \
           #5: Response 1: {orig_dict['Q5']} and Response 2: {responses[4]} \
           #6: Response 1: {orig_dict['Q6']} and Response 2: {responses[5]} \
           #7: Response 1: {orig_dict['Q7']} and Response 2: {responses[6]} \
           #8: Response 1: {orig_dict['Q8']} and Response 2: {responses[7]} \
           #9: Response 1: {orig_dict['Q9']} and Response 2: {responses[8]} \
           #10 Response 1: {orig_dict['Q10']} and Response 2: {responses[0]} \
           """

          response2 = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
              {"role": "user", "content": f"{context2}"},
                ], seed=seed
            )
          orig_dict = response2.choices[0].message.content.split(',') + choices_list
          # orig_dict = orig_dict + choices_list[:-1]#.update({'gender':choices_list[0],
          #                   # 'age':choices_list[1],
          #                   # 'race':choices_list[2],
          #                   # 'dist':choices_list[3]})
          dict_list.append(orig_dict)
          pd.DataFrame(dict_list).to_csv('updated_ratings.csv')
        except:
          print('failed')
          continue

In [ ]:
#Experiment 2: Prompt Creation and Evaluation with AutoPrompt

#PART 1: Generate Persona
# Use a pipeline as a high-level helper
for _ in range(100):
    choices_list = [choice(list(age_dist.keys()), p=list(age_dist.values())),
                      choice(list(race_dist.keys()), p=list(race_dist.values())),
                      choice(list(gender_dist.keys()), p=list(gender_dist.values())),
                      choice(list(geospatial_dist.keys()), p=list(geospatial_dist.values()))
                      ]
    n_texts = random.randrange(1, content_variability_max)
    seed = int(random.random()*(10000))

    context_simulation = f"Let's play a game. You are a {choices_list[0]} \
                          {choices_list[1]} {choices_list[2]} \
                          living in {choices_list[3]}. \
                          Complete the following 4 tasks,output as .json:\
                          Task #1: Create a backstory for yourself and print it. The json key for task #1 should be ``backstory''. \
                          Task #2: \
                          Give me {n_texts} text messages, emails, and letters from you (based on your backstory), each at most {max_char} characters long. The json key for task #2 should be ``data'' and the value should be a list where each entry is one of the created text messages, emails, or letters\
                          Task #3: think of a question that you would like to ask ChatGPT based on a small subset of your messages, emails, and letters. The json key for task #3 should be ``question'' \
                          Task #4 create an initial prompt for this question that is less than 10 words. The json key for task #4 should be ``prompt''"

    response1 = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": f"{context_simulation}"},
          ], seed=seed
      )


    #Part 2: Make the vector of similarities and take the indexes of the top 10% similar documents
    threshold_pct = 0.80
    try:
      with open(f"""persona_{'_'.join(choices_list)}_{seed}.json""", 'w') as f:
          json.dump(response1.choices[0].message.content, f)
      input_list = json.loads(response1.choices[0].message.content)['data'] + [json.loads(response1.choices[0].message.content)['prompt'] for _ in range(len(json.loads(response1.choices[0].message.content)['data']))]
      sentences = input_list
      model = SentenceTransformer('thenlper/gte-base')
      embeddings = model.encode(sentences)
      similarity_vec = [cos_sim(embeddings[i], embeddings[len(input_list)//2+i])[0] for i in range(len(input_list)//2)]
      similarity_vec = [float(x[0]) for x in similarity_vec]
      selected_context_list = []
      for idx, bools in enumerate(pd.Series(similarity_vec) > 0.8):
        if bools:
          selected_context_list.append(sentences[idx])
      selected_context =''.join(selected_context_list)
      if len(selected_context_list) != 0:
        selected_context = selected_context_list[similarity_vec.argmax()]

      #Part 3: Get responses with prompt and prompt with context
      resp_orig = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "user", "content": f"{json.loads(response1.choices[0].message.content)['prompt']}"},
            ], seed=seed
        )
      resp_context = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"{json.loads(response1.choices[0].message.content)['prompt']}" + '. Here is some background information about me that you can choose to include in your response or ignore entirely:' + selected_context.replace("Email", "").replace("Text", "").replace("Message", "").replace("Letter", "").replace("Dear", "")},
            ], seed=seed
        )

      resp3_context = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"{json.loads(response1.choices[0].message.content)['prompt']}" + '. Here is a bit more about me: I am a ' + f"{choices_list[0]} \
                          {choices_list[1]} {choices_list[2]} \
                          living in {choices_list[3]}."},
            ], seed=seed
        )

      response_orig = resp_orig.choices[0].message.content
      response_context = (resp_context.choices[0].message.content)
      resp_3_context = (resp3_context.choices[0].message.content)

      response2 = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "user", "content": f"{context_simulation}"},
          {"role": "assistant", "content": response1.choices[0].message.content},
          {"role": "user", "content":f"Rank how helpful and personal these responses are to your question from 0 (worst) to 10 (best) where most responses are average (5) as three numbers separated by a comma (e.g.: [4,4]) : RESPONSE #1: {response_orig}, RESPONSE #2: {response_context}, RESPONSE #3: {resp_3_context}."},
          ], seed=seed
        )
      vals_list = (response2.choices[0].message.content).split(',')
      # print(resp_list)
      resp_list.append({'age':choices_list[0], 'race':choices_list[1], 'gender':choices_list[2], 'response2':response2, 'geospatial':choices_list[3], 'resp_orig':response_orig, 'resp_context':response_context, 'resp_3': resp_3_context, 'gpt_ranking_orig': vals_list[0], 'gpt_ranking_context': vals_list[1], 'gpt_ranking_person': vals_list[2], 'n_texts':n_texts, 'sel_texts':len(selected_context_list)})
      pd.DataFrame(resp_list).to_csv('run_results_final_new.csv')
      # print(pd.DataFrame(resp_list).iloc[:,6:])
    except:
      continue




In [ ]:
#Experiment 3: Variation on the prompt

for _ in range(n):
    n_texts = random.randrange(1, content_variability_max)
    seed = int(random.random()*(10000))

    context_simulation = f"Let's play a game. You are a {choices_list[0]} \
                          {choices_list[1]} {choices_list[2]} \
                          living in {choices_list[3]}. \
                          Complete the following 4 tasks,output as .json:\
                          Task #1: Create a backstory for yourself and print it. The json key for task #1 shouuld be ``backstory''. \
                          Task #2: \
                          Give me {n_texts} text messages, emails, and letters from you (based on your backstory), each at most {max_char} characters long. The json key for task #2 should be ``data'' and the value should be a list where each entry is one of the created text messages, emails, or letters\
                          Task #3: think of a question that you would like to ask ChatGPT based on a small subset of your messages, emails, and letters. The json key for task #3 should be ``question'' \
                          Task #4 create an initial prompt for this question that is less than 10 words. The json key for task #4 should be ``prompt''"

    response1 = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": f"{context_simulation}"},
          ], seed=seed
      )


    #Part 2: Make the vector of similarities and take the indexes of the top 10% similar documents
    threshold_pct = 0.80
    try:
      with open(f"""persona_{'_'.join(choices_list)}_{seed}.json""", 'w') as f:
          json.dump(response1.choices[0].message.content, f)
      input_list = json.loads(response1.choices[0].message.content)['data'] + [json.loads(response1.choices[0].message.content)['prompt'] for _ in range(len(json.loads(response1.choices[0].message.content)['data']))]
      sentences = input_list
      model = SentenceTransformer('thenlper/gte-base')
      embeddings = model.encode(sentences)
      similarity_vec = [cos_sim(embeddings[i], embeddings[len(input_list)//2+i])[0] for i in range(len(input_list)//2)]
      similarity_vec = [float(x[0]) for x in similarity_vec]
      selected_context_list = []
      for idx, bools in enumerate(pd.Series(similarity_vec) > 0.8):
        if bools:
          selected_context_list.append(sentences[idx])
      selected_context =''.join(selected_context_list)

      #Part 3: Get responses with prompt and prompt with context
      resp_orig = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "user", "content": f"{json.loads(response1.choices[0].message.content)['prompt']}"},
            ], seed=seed
        )
      resp_context = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"{json.loads(response1.choices[0].message.content)['prompt']}" + '. Here is some background information about me that you can choose to include in your response or ignore entirely:' + selected_context.replace("Email", "").replace("Text", "").replace("Message", "").replace("Letter", "").replace("Dear", "")},
            ], seed=seed
        )

      response_orig = resp_orig.choices[0].message.content
      response_context = (resp_context.choices[0].message.content)
      resp_list_2 = [response_orig, response_context]
      first  = random.randint(0, 1)

      response2 = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "user", "content": f"{context_simulation}"},
          {"role": "assistant", "content": response1.choices[0].message.content},
          {"role": "user", "content":f"Rank how helpful and personal these responses are to your question from 0 (worst) to 10 (best) where most responses are average (5) as two numbers separated by a comma (e.g.: [4,4]) : RESPONSE #1: {resp_list_2[first]}, RESPONSE #2: {resp_list_2[~first]}."},
          ], seed=seed
        )
      vals_list = (response2.choices[0].message.content).split(',')
      print(resp_list)
      resp_list.append({'age':choices_list[0], 'race':choices_list[1], 'gender':choices_list[2], 'response2':response2, 'geospatial':choices_list[3], 'resp_orig':response_orig, 'resp_context':response_context, 'gpt_ranking_orig': vals_list[first], 'gpt_ranking_context': vals_list[~first], 'n_texts':n_texts, 'sel_texts':len(selected_context_list)})
      pd.DataFrame(resp_list).to_csv('run_results_final_new.csv')
      print(pd.DataFrame(resp_list).iloc[:,6:])
    except:
      continue